# Sistemas lineares

* Cálculo de inversa ($A^{-1}$)
* Condicionamento de matriz

https://www.ufrgs.br/reamat/CalculoNumerico/livro-py/sdsl-condicionamento_de_sistemas_lineares.html

## Cálcula da inversa

Uma propriedade importante para calcular a inversa de uma matriz vem do fato que, ao resolvermos o sistema:

$
A \cdot x  = 
\begin{bmatrix} 1 \\ 0 \\ 0 \\ \vdots \\ 0 \end{bmatrix} 
$

encontramos a primeira coluna de $A^{-1}$.

De forma análoga, ao usarmos a $n$-ésima coluna da matriz identidade, achamos a $n$-ésima coluna da inversa de $A$.

In [87]:
# Exemplo do calculo da inversa:
A = np.array([[1,1,1], [2,1,-1], [2,-1,1]])

I = np.identity(3)
I1 = I[0,:]
I2 = I[1,:]
I3 = I[2,:]

x1 = np.linalg.solve(A,I1).reshape((3,1))
x2 = np.linalg.solve(A,I2).reshape((3,1))
x3 = np.linalg.solve(A,I3).reshape((3,1))

inversa = x1
inversa = np.concatenate((inversa , x2), axis=1)
inversa = np.concatenate((inversa , x3), axis=1)

print(inversa, '\n')
print(np.linalg.inv(A))

[[ 0.     0.25   0.25 ]
 [ 0.5    0.125 -0.375]
 [ 0.5   -0.375  0.125]] 

[[ 0.     0.25   0.25 ]
 [ 0.5    0.125 -0.375]
 [ 0.5   -0.375  0.125]]


In [97]:
def lu(A):
    n = np.shape(A)[0]
    L = np.identity(n)
    U = np.copy(A)

    for PIVO in range(n):
        for LIN in range(PIVO+1, n):
            C = U[LIN,PIVO]/U[PIVO,PIVO]
            L[LIN,PIVO] = C
            U[LIN,:] = U[LIN,:] - C*U[PIVO,:]
    return L,U

# https://johnfoster.pge.utexas.edu/numerical-methods-book/LinearAlgebra_LU.html
def forward_substitution(L, b):
    #Get number of rows
    n = L.shape[0]
    
    #Allocating space for the solution vector
    y = np.zeros_like(b, dtype=np.double);
    
    #Here we perform the forward-substitution.  
    #Initializing  with the first row.
    y[0] = b[0] / L[0, 0]
    
    #Looping over rows in reverse (from the bottom  up),
    #starting with the second to last row, because  the 
    #last row solve was completed in the last step.
    for i in range(1, n):
        y[i] = (b[i] - np.dot(L[i,:i], y[:i])) / L[i,i]
        
    return y

def back_substitution(U, y):
    #Number of rows
    n = U.shape[0]
    
    #Allocating space for the solution vector
    x = np.zeros_like(y, dtype=np.double);

    #Here we perform the back-substitution.  
    #Initializing with the last row.
    x[-1] = y[-1] / U[-1, -1]
    
    #Looping over rows in reverse (from the bottom up), 
    #starting with the second to last row, because the 
    #last row solve was completed in the last step.
    for i in range(n-2, -1, -1):
        x[i] = (y[i] - np.dot(U[i,i:], x[i:])) / U[i,i]
        
    return x

In [101]:
# Exemplo de cálculo de inversa utilizando a fatoração LU
A = np.array([[1,1,1], [2,1,-1], [2,-1,1]])

# a função LU só precisa ser chamada uma única vez:
L,U = lu(A)

n = A.shape[0]
I = np.identity(n)
inversa = np.empty((n,0))

# observe que o laço principal só utiliza as chamadas para as funções forward_substitution
# e back_substitution, que requerem poucos cálculos.
for i in range(n):
    In = I[i,:]
    y = forward_substitution(L, In)
    x = back_substitution(U, y).reshape((n,1))
    inversa = np.concatenate((inversa, x), axis=1)

print(inversa, '\n')
print(A @ inversa)

[[ 0.     0.25   0.25 ]
 [ 0.5    0.125 -0.375]
 [ 0.5   -0.375  0.125]] 

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


## Condicionamento de uma matriz

In [35]:
import numpy as np
A1 = np.array( [ [71, 41], [51, 30] ] )
A2 = np.array( [ [71, 41], [52, 30] ] )

b1 = np.array( [100.0, 70.0] )
b2 = np.array( [100.4, 69.3] )

x1 = np.linalg.solve(A1,b1)
x2 = np.linalg.solve(A2,b1)
x3 = np.linalg.solve(A2,b2)

print(x1, '\n',x2, '\n', x3)

[ 3.33333333 -3.33333333] 
 [-65. 115.] 
 [-85.35 150.25]


Analise o que aconteceu no exemplo anterior:

Uma pequena variação na entrada ($A$ ou $b$) produziu uma grande diferença no valor de $x$.

Nestes casos, dizemos que o sistema é **mal condicionado**.

Considere um sistema linear:

$Ax = y$

Suponha que tenhamos um erro na entrada de $\delta_x$, então teremos:

$A(x + \delta_x) = y + \delta_y$, onde $\delta_y$ é o erro associado à $\delta_x$. 

Teremos então que:

$A\delta_x = \delta_y$

Estamos interessados em relacionar o erro relativo em $x$ e o erro relativo em $y$:

$
\dfrac{\lVert \delta_x \rVert / \lVert x \rVert }{\lVert \delta_y \rVert / \lVert y \rVert }
$

Sabendo que:

$
\lVert \delta_x \rVert = \lVert A^{-1}\delta_y \rVert 
$ e

$
\lVert y \rVert = \lVert Ax \rVert 
$ 

temos que:

$
\dfrac{\lVert \delta_x \rVert / \lVert x \rVert }{\lVert \delta_y \rVert / \lVert y \rVert } \leq 
\lVert A \rVert \cdot \lVert A^{-1} \rVert
$


O número de condicionamento de uma matriz é definido como:

$
Cond(A) = \lVert A \rVert \cdot \lVert A^{-1} \rVert
$

e está relacionado à sensibilidade a variações de uma matriz, ou seja, como os erros de arredondamento se propagam de forma mais ou menos relevante.

In [95]:
import numpy as np

A2 = np.array( [ [71, 41], [52, 30] ] )

np.linalg.cond(A2, p=np.inf)

6887.999999999792